This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# First, import the relevant modules
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize

# Store the API key as a string - according to PEP8, constants are always named in all upper case
with open('credentials.json') as key:
    data = json.load(key)
    API_KEY = data['API_KEY']

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key={}".format(API_KEY)
requests.get(url).json()['dataset_data']['data'][0]

['2019-01-02', None, 70.85, 68.25, 69.85, None, None, None, None, None, None]

In [ ]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
requests.get(url).json()

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

##### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [4]:
meditec_equities = requests.get(url).json()['dataset_data']['data']

##### 2. Convert the returned JSON object into a Python dictionary.

In [5]:
# creates dictionary of 2017 data from JSON object
mt_2017_equities_dict = {}
for entry in meditec_equities:
    if '2017' in entry[0]:
        mt_2017_equities_dict[entry[0]] = {}
        mt_2017_equities_dict[entry[0]]['Open'] = entry[1]
        mt_2017_equities_dict[entry[0]]['High'] = entry[2]
        mt_2017_equities_dict[entry[0]]['Low'] = entry[3]
        mt_2017_equities_dict[entry[0]]['Close'] = entry[4]
        mt_2017_equities_dict[entry[0]]['Change'] = entry[5]
        mt_2017_equities_dict[entry[0]]['Traded Volume'] = entry[6]
        mt_2017_equities_dict[entry[0]]['Turnover'] = entry[7]
        mt_2017_equities_dict[entry[0]]['Last Price of the Day'] = entry[8]
        mt_2017_equities_dict[entry[0]]['Daily Traded Units'] = entry[9]
        mt_2017_equities_dict[entry[0]]['Daily Turnover'] = entry[10]

In [14]:
# now let's make a dataframe out of the dictionary to answer the questions
df_eqts17 = pd.DataFrame.from_dict(mt_2017_equities_dict).transpose()
df_eqts17.head()

,Change,Close,Daily Traded Units,Daily Turnover,High,Last Price of the Day,Low,Open,Traded Volume,Turnover
2017-12-29,NaN,51.76,NaN,NaN,51.94,NaN,51.45,51.76,34640.0,1792304.0
2017-12-28,NaN,51.60,NaN,NaN,51.82,NaN,51.43,51.65,40660.0,2099024.0
2017-12-27,NaN,51.82,NaN,NaN,51.89,NaN,50.76,51.45,57452.0,2957018.0
2017-12-22,NaN,51.32,NaN,NaN,51.50,NaN,50.92,51.05,71165.0,3641949.0
2017-12-21,NaN,51.40,NaN,NaN,51.52,NaN,50.90,51.16,120649.0,6179433.0


##### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [25]:
open_min = df_eqts17['Open'].min()
open_max = df_eqts17['Open'].max()
print('Highest opening price (2017): ${:.2f}'.format(open_max))
print('Lowest opening price (2017): ${:.2f}'.format(open_min))

Highest opening price (2017): $53.11
Lowest opening price (2017): $34.00


##### 4. What was the largest change in any one day (based on High and Low price)?

In [16]:
# first lets create a dataframe from the json object for the meditec equity for all years
df_eqts = json_normalize(requests.get(url).json()['dataset_data'], 'data')
df_eqts.columns = requests.get(url).json()['dataset_data']['column_names']
df_eqts['Date'] = pd.to_datetime(df_eqts['Date'], format='%Y/%m/%d', utc=True)
df_eqts.set_index('Date', inplace=True)
df_eqts.head()

,Open,High,Low,Close,Change,Traded Volume,Turnover,Last Price of the Day,Daily Traded Units,Daily Turnover
Date,,,,,,,,,,
2019-01-04 00:00:00+00:00,NaN,71.00,68.80,70.75,NaN,NaN,NaN,None,None,NaN
2019-01-03 00:00:00+00:00,NaN,69.35,67.75,68.50,NaN,NaN,NaN,None,None,NaN
2019-01-02 00:00:00+00:00,NaN,70.85,68.25,69.85,NaN,NaN,NaN,None,None,NaN
2018-12-28 00:00:00+00:00,NaN,68.80,67.75,68.30,NaN,NaN,NaN,None,None,NaN
2018-12-27 00:00:00+00:00,NaN,70.65,67.85,68.25,NaN,NaN,NaN,None,None,NaN


In [13]:
largest_daily_change = (df_eqts18['High'] - df_eqts18['Low']).max()
print('Largest change between high and low price in a single day (all years): ${:.2f}'.format(largest_daily_change))

Largest change between high and low price in a single day (all years): $7.70


##### 5. What was the largest change between any two days (based on Closing Price)?

In [22]:
largest_bidaily_change = abs(mt_equities_df['Close']-mt_equities_df['Close'].shift(1)).max()
print('Largest change in closing price between two days (all years): ${:.2f}'.format(largest_bidaily_change))

Largest change in closing price between two days (all years): $7.00


##### 6. What was the average daily trading volume during this year (2018)?

In [23]:
df_eqts18 = mt_equities_df.loc['2018']
daily_avg = df_eqts18['Traded Volume'].sum()/len(df_eqts18['Traded Volume'])
print('The average daily trading volume for 2018 is: ${:.2f}'.format(daily_avg))

The average daily trading volume for 2018 is: $95563.47


##### 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [24]:
med_vol = mt_2018_equities_df['Traded Volume'].median()
print('The median trading volume for 2018 is: ${:.2f}'.format(med_vol))

The median trading volume for 2018 is: $85255.50
